In [0]:
%sql
USE CATALOG main;
CREATE SCHEMA IF NOT EXISTS silver;
USE SCHEMA silver;


Create a Sample Large Table (Unoptimized)

In [0]:
%sql
CREATE OR REPLACE TABLE sales_raw
USING DELTA
AS
SELECT
  id,
  CAST(rand() * 10000 AS INT)        AS customer_id,
  CAST(rand() * 100 AS INT)          AS product_id,
  DATE_ADD('2023-01-01', CAST(rand() * 365 AS INT)) AS order_date,
  CAST(rand() * 1000 AS DOUBLE)      AS amount
FROM range(10000000);   -- 10 million rows


num_affected_rows,num_inserted_rows


In [0]:
%sql
EXPLAIN FORMATTED
SELECT SUM(amount)
FROM sales_raw
WHERE order_date = '2023-06-15'
  AND customer_id = 1234;


plan
"== Physical Plan == AdaptiveSparkPlan (10) +- == Initial Plan == ColumnarToRow (9) +- PhotonResultStage (8) +- PhotonAgg (7) +- PhotonShuffleExchangeSource (6) +- PhotonShuffleMapStage (5) +- PhotonShuffleExchangeSink (4) +- PhotonAgg (3) +- PhotonProject (2) +- PhotonScan parquet main.silver.sales_raw (1) (1) PhotonScan parquet main.silver.sales_raw Output [3]: [customer_id#13591, order_date#13593, amount#13594] DictionaryFilters: [(customer_id#13591 = 1234), (order_date#13593 = 2023-06-15)] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-cpsfj/uc/16bae815-1321-420c-b229-b9b2bdee15a6/53db5cc4-66ce-49f0-9a63-f5af98f2bca7/__unitystorage/catalogs/1d75f74d-9803-4f40-b371-cdd8e1835f5a/tables/d43b459c-8b8e-4640-bd77-ce8b791cec08] ReadSchema: struct RequiredDataFilters: [isnotnull(order_date#13593), isnotnull(customer_id#13591), (order_date#13593 = 2023-06-15), (customer_id#13591 = 1234)] (2) PhotonProject Input [3]: [customer_id#13591, order_date#13593, amount#13594] Arguments: [amount#13594] (3) PhotonAgg Input [1]: [amount#13594] Arguments: [partial_sum(amount#13594) AS sum#13624], [sum#13623], [sum#13624], false (4) PhotonShuffleExchangeSink Input [1]: [sum#13624] Arguments: SinglePartition (5) PhotonShuffleMapStage Input [1]: [sum#13624] Arguments: ENSURE_REQUIREMENTS, [id=#8503] (6) PhotonShuffleExchangeSource Input [1]: [sum#13624] (7) PhotonAgg Input [1]: [sum#13624] Arguments: [finalmerge_sum(merge sum#13624) AS sum(amount)#13596], [sum(amount)#13596], [sum(amount)#13596 AS SUM(amount)#13597], true (8) PhotonResultStage Input [1]: [SUM(amount)#13597] (9) ColumnarToRow Input [1]: [SUM(amount)#13597] (10) AdaptiveSparkPlan Output [1]: [SUM(amount)#13597] Arguments: isFinalPlan=false == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = full = sales_raw"


Analyze Query Plan (Before Optimization)

In [0]:
%sql
SELECT SUM(amount)
FROM sales_raw
WHERE order_date = '2023-06-15'
  AND customer_id = 1234;


SUM(amount)
1767.252468870618


View query plan

In [0]:
%sql
EXPLAIN FORMATTED
SELECT SUM(amount)
FROM sales_raw
WHERE order_date = '2023-06-15'
  AND customer_id = 1234;


plan
"== Physical Plan == AdaptiveSparkPlan (10) +- == Initial Plan == ColumnarToRow (9) +- PhotonResultStage (8) +- PhotonAgg (7) +- PhotonShuffleExchangeSource (6) +- PhotonShuffleMapStage (5) +- PhotonShuffleExchangeSink (4) +- PhotonAgg (3) +- PhotonProject (2) +- PhotonScan parquet main.silver.sales_raw (1) (1) PhotonScan parquet main.silver.sales_raw Output [3]: [customer_id#13733, order_date#13735, amount#13736] DictionaryFilters: [(customer_id#13733 = 1234), (order_date#13735 = 2023-06-15)] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-cpsfj/uc/16bae815-1321-420c-b229-b9b2bdee15a6/53db5cc4-66ce-49f0-9a63-f5af98f2bca7/__unitystorage/catalogs/1d75f74d-9803-4f40-b371-cdd8e1835f5a/tables/d43b459c-8b8e-4640-bd77-ce8b791cec08] ReadSchema: struct RequiredDataFilters: [isnotnull(order_date#13735), isnotnull(customer_id#13733), (order_date#13735 = 2023-06-15), (customer_id#13733 = 1234)] (2) PhotonProject Input [3]: [customer_id#13733, order_date#13735, amount#13736] Arguments: [amount#13736] (3) PhotonAgg Input [1]: [amount#13736] Arguments: [partial_sum(amount#13736) AS sum#13755], [sum#13754], [sum#13755], false (4) PhotonShuffleExchangeSink Input [1]: [sum#13755] Arguments: SinglePartition (5) PhotonShuffleMapStage Input [1]: [sum#13755] Arguments: ENSURE_REQUIREMENTS, [id=#8748] (6) PhotonShuffleExchangeSource Input [1]: [sum#13755] (7) PhotonAgg Input [1]: [sum#13755] Arguments: [finalmerge_sum(merge sum#13755) AS sum(amount)#13738], [sum(amount)#13738], [sum(amount)#13738 AS SUM(amount)#13739], true (8) PhotonResultStage Input [1]: [SUM(amount)#13739] (9) ColumnarToRow Input [1]: [SUM(amount)#13739] (10) AdaptiveSparkPlan Output [1]: [SUM(amount)#13739] Arguments: isFinalPlan=false == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = full = sales_raw"


Create a Partitioned Table (Optimization #1)

In [0]:
%sql
CREATE OR REPLACE TABLE sales_partitioned
USING DELTA
PARTITIONED BY (order_date)
AS
SELECT * FROM sales_raw;


num_affected_rows,num_inserted_rows


Re-run the same query

In [0]:
%sql
EXPLAIN FORMATTED
SELECT SUM(amount)
FROM sales_partitioned
WHERE order_date = '2023-06-15'
  AND customer_id = 1234;


plan
"== Physical Plan == AdaptiveSparkPlan (6) +- == Initial Plan == ColumnarToRow (5) +- PhotonResultStage (4) +- PhotonAgg (3) +- PhotonProject (2) +- PhotonScan parquet main.silver.sales_partitioned (1) (1) PhotonScan parquet main.silver.sales_partitioned Output [3]: [customer_id#14050, amount#14053, order_date#14052] DictionaryFilters: [(customer_id#14050 = 1234)] Location: PreparedDeltaFileIndex [s3://dbstorage-prod-cpsfj/uc/16bae815-1321-420c-b229-b9b2bdee15a6/53db5cc4-66ce-49f0-9a63-f5af98f2bca7/__unitystorage/catalogs/1d75f74d-9803-4f40-b371-cdd8e1835f5a/tables/fc8ae631-9e04-410b-a799-90a785a60596] PartitionFilters: [isnotnull(order_date#14052), (order_date#14052 = 2023-06-15)] ReadSchema: struct RequiredDataFilters: [isnotnull(customer_id#14050), (customer_id#14050 = 1234)] (2) PhotonProject Input [3]: [customer_id#14050, amount#14053, order_date#14052] Arguments: [amount#14053] (3) PhotonAgg Input [1]: [amount#14053] Arguments: [sum(amount#14053)], [sum(amount)#14055], [sum(amount)#14055 AS SUM(amount)#14056], true (4) PhotonResultStage Input [1]: [SUM(amount)#14056] (5) ColumnarToRow Input [1]: [SUM(amount)#14056] (6) AdaptiveSparkPlan Output [1]: [SUM(amount)#14056] Arguments: isFinalPlan=false == Photon Explanation == The query is fully supported by Photon. == Optimizer Statistics (table names per statistics state) == missing = partial = sales_partitioned full = Corrective actions: consider running the following command on all tables with missing or partial statistics ANALYZE TABLE COMPUTE STATISTICS FOR ALL COLUMNS"


Apply ZORDER (Optimization #2)

In [0]:
%sql
OPTIMIZE sales_partitioned
ZORDER BY (customer_id);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 365, List(minCubeSize(107374182400), List(0, 0), List(365, 167920876), 0, List(0, 0), 0, null), null, 0, 0, 365, 365, false, 0, 0, 1768888598518, 1768888602846, 8, 0, null, List(0, 0), null, 5, 5, 0, 0, null)"


Benchmark Performance (Before vs After)

In [0]:
%sql
SELECT SUM(amount)
FROM sales_partitioned
WHERE order_date = '2023-06-15'
  AND customer_id = 1234;


SUM(amount)
1767.252468870618


Validate Optimization (Delta Metadata)

In [0]:
%sql
DESCRIBE DETAIL sales_partitioned;


format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,f2fa3db7-ff01-40e0-817c-808430cd099b,main.silver.sales_partitioned,null,,2026-01-20T05:55:03.034Z,2026-01-20T05:55:16.000Z,List(order_date),List(),365,167920876,Map(delta.enableDeletionVectors -> true),3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false
